In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
from implicit.als import AlternatingLeastSquares


In [ ]:
read_file = pd.read_csv (r'C:\Users\Johan\Downloads\dataset_TIST2015\dataset_TIST2015_POIs.txt', names=["Pid", "Latitude", "Longtitude", "Category", "Country"], sep='\t')
read_file.to_csv (r'C:\Users\Johan\Documents\POI_dataset\TIST2015_POIs.csv', index=None)

In [ ]:
read_file = pd.read_csv (r'C:\Users\Johan\Downloads\dataset_TIST2015\dataset_TIST2015_Checkins.txt', names=["Uid","Pid","Timestamp", "TimezoneOffset"], sep='\t')
read_file.to_csv (r'C:\Users\Johan\Documents\POI_dataset\TIST2015_Checkins.csv', index=None)

In [ ]:
def read_data(path):
    files = {}
    for filename in path.glob('*'):
        if filename.suffix == '.csv':
            files[filename.stem] = pd.read_csv(filename)
        elif filename.suffix == '.txt':
            if filename.stem == "TIST2015_POIs":
                columns = ["Pid", "Latitude", "Longtitude", "Category", "Country"]
            elif filename.stem == "TIST2015_POIs_Checkins":
                columns = ["Uid","Pid","Timestamp", "TimezoneOffset"]
            else:
                continue
            data = pd.read_csv(filename, names=columns, engine='python')
            files[filename.stem] = data

    return files['TIST2015_POIs'], files['TIST2015_Checkins']

In [ ]:
# pick one of the available folders
pois, checkins = read_data(Path(r'C:\Users\Johan\Documents\POI_dataset'))


In [ ]:
users_subset = user_checkins_counts['Uid'].unique()
len(users_subset)
checkins.sample(users_subset, 1000)
##users_subset=checkins.loc[checkins['Uid'] <= 1000, 'Pid']
##users_subset


##checkins.head()

In [ ]:
checkins["Uid"].plot(kind='hist')


In [ ]:
checkins.describe(include=object)

In [ ]:
checkins["Uid"].value_counts()


In [ ]:
user_checkins_counts = checkins.groupby('Uid')['Pid'].count()

In [ ]:
# Create a subset of a dataset with size n users
def create_checkin_subset(checkins_table, n):
    subset = np.array([[0,0,0,0],[0,0,0,0]])
    checkins_100_users = checkins_table['Uid'].sample(n=n, random_state=1)
    for i in checkins_100_users:
        subset = np.append(subset, np.asarray(checkins_table[checkins_table['Uid'] == i].values), 0)
    return pd.DataFrame(subset[2::], columns=['Uid', 'Pid', 'Timestamp', 'TimezoneOffset'])

In [ ]:
# Initialize the subset method
checkins_100_users = create_checkin_subset(checkins, 50)

In [ ]:
read_file = pd.DataFrame(checkins_100_users)
read_file.to_csv (r'C:\Users\Johan\Documents\POI_dataset\TIST2015_Checkins_50.csv', index=None)

In [ ]:
checkins_100_users
##points_checkins_counts = checkins.groupby('Pid')['Uid'].count()

In [ ]:
# Merge pois and checkins to one Dataframe
checkin_data = checkins_100_users.merge(pois, on='Pid')

In [ ]:
checkin_data

In [ ]:
# create a matrix of Uid and Pid
df = checkin_data.set_index('Uid').Pid.str.get_dummies(',')

In [ ]:
df

In [ ]:
df.stack().reset_index().rename(columns={'Uid': 'user_id', 'Pid': 'poi_id', 0:'label'})
df

In [ ]:
import scipy

dftomatrix = scipy.sparse.csr_matrix(df.values)

In [ ]:
pd.DataFrame(dftomatrix)

In [ ]:
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares(factors=64, regularization=0.05)
model.fit(2 * dftomatrix)

In [ ]:
userid = 5
ids, scores = model.recommend(userid, dftomatrix[userid], N=10)

In [ ]:
# Use pandas to display the output in a table, pandas isn't a dependency of implicit otherwise

pd.DataFrame({"points": pois[ids], "score": scores, "already_visited": np.in1d(ids, checkins_100_users[userid].indices)})

In [ ]:
pd.DataFrame(df)

In [ ]:
# Remove Uid Duplicates in matrix
df = df.groupby('Uid').max()
print(df)

In [ ]:
Users = checkins['Uid'].unique()
print(len(Users))
Ncheckins = np.zeros(len(Users))
for user,i in zip(Users, range(len(Users))):
    Ncheckins[i] = np.where(checkins['Uid'] == Users)

In [ ]:
plt.hist(user_checkins_counts)
plt.show()
##user_checkins_counts.plot(kind='hist', x=user_checkins_counts, y=points_checkins_counts)

In [ ]:
def tabular_preview(checkins, n=15):
    """Creates a cross-tabular view of users vs movies."""

    user_groups = checkins.groupby('Uid')['Pid'].count()
    top_users = user_groups.sort_values(ascending=False)[:15]

    poi_groups = checkins.groupby('Pid')['Uid'].count()
    top_pois = poi_groups.sort_values(ascending=False)[:15]

    top = (
        checkins.
        join(top_users, rsuffix='_r', how='inner', on='Uid').
        join(top_pois, rsuffix='_r', how='inner', on='Pid'))

    plt.hist(top)
    plt.show()

In [ ]:
tabular_preview(checkin_data)

In [ ]:
A=nx.linalg.graphmatrix.adjacency_matrix(checkins_100_users).todense()
L=nx.linalg.laplacianmatrix.laplacian_matrix(checkins_100_users).todense()

In [ ]:
A=

